# train_simcse.ipynb
Toy PyTorch SimCSE training (CPU-friendly)

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import pandas as pd
import os

class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=64):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        encoded = self.tokenizer(text, truncation=True, padding='max_length',
                                 max_length=self.max_len, return_tensors='pt')
        return {key: val.squeeze(0) for key,val in encoded.items()}

class SimCSE(nn.Module):
    def __init__(self, pretrained_model="bert-base-uncased"):
        super().__init__()
        self.encoder = BertModel.from_pretrained(pretrained_model)
    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.pooler_output

# Load dataset
df = pd.read_csv("dataset/obituaries_sample.csv")
texts = df['text'].astype(str).tolist()[:200]  # small subset for CPU demo

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
dataset = TextDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimCSE().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

def simcse_loss(z):
    # z: [batch, dim]
    z_norm = z / z.norm(dim=1, keepdim=True)
    sim = torch.matmul(z_norm, z_norm.T)
    labels = torch.arange(z.size(0)).to(device)
    loss_fct = nn.CrossEntropyLoss()
    loss = loss_fct(sim, labels)
    return loss

model.train()
for epoch in range(1):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        z = model(input_ids, attention_mask)
        loss = simcse_loss(z)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch done, loss:", loss.item())

os.makedirs("saved_model", exist_ok=True)
torch.save(model.state_dict(), "saved_model/encoder.pt")
print("Model saved at saved_model/encoder.pt")
